In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, IterableDataset
import chess
from time import time
import tqdm

HEADERS = ("bitmaps", "movePlayed", "validMoves")
BATCH_SIZE = 64

In [2]:
import torch.nn as nn
import torch.nn.functional as F
class PositionEvaluatorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(13, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x))  # Output between -1 and 1
    

class ChessBotNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # 13 channels: 12 pieces + side to play (tensor[true's|false's])
        self.conv1 = nn.Conv2d(13,64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(8 * 8 * 128, 256)
        self.fc2 = nn.Linear(256, 64)
        self.relu = nn.ReLU()

        # Initialize weights
        nn.init.kaiming_uniform_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight, nonlinearity='relu')
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)


    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # Output raw logits
        return x

class _CompleteChessBotNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # 13 channels: 12 pieces + side to play (tensor[true's|false's])
        self.conv1 = nn.Conv2d(13, 64, kernel_size=3, padding='same')
        self.batchnorm1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding='same')
        self.batchnorm2 = nn.BatchNorm2d(128)

        self.fc1 = nn.Linear(8 * 8 * 128, 256) # 8 * 8 (num of squares) * 128 (num of channels)
        self.fc2 = nn.Linear(256, 64 * 63) # (Choose 2 squares from the board where the order matters)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.flatten = nn.Flatten()

        # Initialize weights
        nn.init.kaiming_uniform_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight, nonlinearity='relu')
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)


    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.batchnorm1(x)

        x = self.relu(self.conv2(x))
        x = self.batchnorm2(x)

        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # Output raw logits
        return x

class CompleteChessBotNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # 13 channels: 12 pieces + side to play (tensor[true's|false's])

        self.inputLayer = nn.Conv2d(13, 64, kernel_size=3, padding='same')
        self.batchnorm0 = nn.BatchNorm2d(64)

        self.conv1 = nn.Conv2d(64, 64, kernel_size=3, padding='same')
        self.batchnorm1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding='same')
        self.batchnorm2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding='same')
        self.batchnorm3 = nn.BatchNorm2d(64)


        self.convLayers = nn.Sequential(
            self.inputLayer,
            self.batchnorm0,
            nn.ReLU(),
            self.conv1,
            self.batchnorm1,
            nn.ReLU(),
            self.conv2,
            self.batchnorm2,
            nn.ReLU(),
            self.conv3,
            self.batchnorm3,
            nn.ReLU()
        )

        self.fc1 = nn.Linear(8 * 8 * 64, 256) # 8 * 8 (num of squares) * 128 (num of channels)
        self.fc2 = nn.Linear(256, 64 * 63) # (Choose 2 squares from the board where the order matters)
        
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

        # Initialize weights
        nn.init.kaiming_uniform_(self.inputLayer.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv3.weight, nonlinearity='relu')
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)


    def forward(self, x):
        x = self.convLayers(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # Output raw logits
        return x
## Idea:
##   One model that answers "best piece to move in this position"
##   Then another model that answers "best square to move piece X to"

# Add normalization after conv
# Switch from relu to sigmoid or smth
# Add more preprocessing by making the tensors there and avoid pre processing before training

In [3]:
import json

piece_to_idx = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
}

def board_to_tensor(board):
    tensor = np.zeros((12, 8, 8), dtype=np.uint8)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            idx = piece_to_idx[piece.symbol()]
            row = 7 - square // 8
            col = square % 8
            tensor[idx, row, col] = 1
    return tensor

def convert_to_array_bitmap(row: str):
    """"
    Converts board into array

    :param str row: Board with side to play;
                        This must represent a np.ndarray[np.int64, shape=(13)] 
    :return: np.ndarray[shape(13, 8, 8), dtype=np.float32]]:
    """
    boards = np.array(json.loads(row), dtype=np.uint64) # shape = (13,)
    array = np.empty((13, 8, 8), dtype=np.uint8)

    # Set pieces boards
    board_int8_view = boards.view(dtype=np.uint8).reshape((13, 8)) # shape = (13, 8)
    board_as_int = np.unpackbits(board_int8_view, axis=1).reshape((13, 8, 8))
    array[:] = board_as_int

    # Set side to play board
    array[12] = np.ones(shape = (1, 8, 8)) if boards[12] == 1 else np.zeros(shape=(1,8,8))
    return array


letters = ["a", "b", "c", "d", "e", "f", "g", "h"]
numbers = list(range(1, 10)) # [1..9]
MOVE_DICTIONARY = {}
cumulative = 0
for i in range(8):
    for j in range(8):
        for k in range(8):
            for w in range (8):
                if (i == k and j == w):
                    cumulative += 1
                    continue
                from_square = f"{letters[i]}{numbers[j]}"
                to_square = f"{letters[k]}{numbers[w]}"
                MOVE_DICTIONARY[f"{from_square}{to_square}"] = (i * 8**3) + (j * 8**2) + (k * 8) + w - cumulative
REVERSE_MOVE_DICTIONARY = {
    value: key for key,value in MOVE_DICTIONARY.items()
}

In [4]:
from typing import Literal
import polars as pl
import random

class ChessEvalDataset(Dataset):
# class ChessEvalDataset(IterableDataset):
    def __init__(self, file: str, model: Literal["pieces", "moves"] = "pieces", load_batch_size = 6_400):
        self.model = model
        self.lazy_dataset = pl.scan_csv(file, has_header=False, new_columns=HEADERS)
        self.batch_size = load_batch_size
        self.feature_col = "bitmaps"
        self.target_col = "movePlayed"
        self.total_rows = self.lazy_dataset.select(pl.len()).collect().item()

        self.cached_batches: dict[int, tuple] = {}
        self.cached_batch_id: int | None = None

    def __len__(self):
        return self.total_rows
    
    def _get_batch(self, batch_id):
        """Load a specific batch of data, wrapped with lru_cache for memory management"""
        # Calculate batch range
        if batch_id == self.cached_batch_id:
            return self.cached_batches[batch_id]
        

        self.cached_batches.pop(self.cached_batch_id, None) # Delete old batch
        
        # Calculate batch range
        start_idx = batch_id * self.batch_size
        end_idx = min((batch_id + 1) * self.batch_size, self.total_rows)
        self.idxs = [i for i in range(end_idx - start_idx)]
        random.shuffle(self.idxs) # Shuffle the indices for the batch
        
        # Fetch only this batch of data using offset and limit
        batch_dataset = (self.lazy_dataset
                    .slice(start_idx, end_idx - start_idx)
                    .collect())
        
        # Process features and target
        features = batch_dataset.select(self.feature_col)
        features = torch.tensor(np.array([convert_to_array_bitmap(bitmaps) for bitmaps in features["bitmaps"]]), dtype=torch.float32)
        
        played_moves = batch_dataset.select(self.target_col).to_numpy()
        targets = torch.tensor(played_moves, dtype=torch.long)

        valid_moves = batch_dataset.select("validMoves")
        valid_moves = torch.tensor(np.array([json.loads(row) for row in valid_moves["validMoves"]], dtype=np.uint64))
        # valid_moves = np.zeros(shape=np.shape(played_moves))


        self.cached_batch_id = batch_id
        self.cached_batches[self.cached_batch_id] = (features, targets, valid_moves)
        
        return self.cached_batches[batch_id]

    def __getitem__(self, idx):
        # Calculate which batch this index belongs to
        batch_id = idx // self.batch_size
        # Get the batch
        features, targets, valid_moves = self._get_batch(batch_id)

        # Get the item from the batch
        idx_in_batch = self.idxs[idx % self.batch_size]  # Get the next index from the shuffled list

        return features[idx_in_batch], targets[idx_in_batch], valid_moves[idx_in_batch]

In [5]:
VALID_MOVE_LOSS = -0.1/1024
INVALID_MOVE_LOSS = +10/1024

piece_to_idx = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
}

def bitmaps_to_board(bitmaps):
    board = chess.Board(fen = "8/8/8/8/8/8/8/8 w KQkq - 0 1")
    for piece_name, piece_idx in piece_to_idx.items():
        for row in range(8):
            for col in range(8):
                if bitmaps[piece_idx][row][col] == 1:
                    piece = chess.Piece(piece_idx, chess.WHITE) if piece_idx < 6 else chess.Piece(piece_idx - 6, chess.BLACK)
                    board.set_piece_at(row * 8 + col, piece)
    return board

# LOSS FUNCTION
cross_entropy_loss = torch.nn.CrossEntropyLoss()
def loss_fn(outputs: torch.Tensor, targets, valid_moves): # targets tensor with twp elements: [targets, possible_moves]
    loss = cross_entropy_loss(outputs, targets)

    played_move = [output.argmax().item() for output in outputs]
    penalization = 0
    invalid_played = False
    for move, valid_moves in zip(played_move, valid_moves):
        num_idx = move // 64
        bit_idx = move % 64
        num = valid_moves[num_idx]

        shift_tensor = torch.tensor(1 << bit_idx, dtype=num.dtype) 

        if num & shift_tensor != 0:
            # invalid_played = True
            penalization += INVALID_MOVE_LOSS
        else:
            penalization += VALID_MOVE_LOSS
            
    # if invalid_played:
        # penalization += INVALID_MOVE_LOSS
    return torch.add(loss, penalization)

In [ ]:
DATASET_PATH = '../dataset/processed/results_ELO_1500.csv'
NUM_EPOCHS = 400

TRAINING_MODE = "pieces" # "pieces" or "moves"
MODEL_WEIGHTS_OUTPUT_PATH = "./models/CompleteModel_new_architecture_skipmoves_FINISHED.pth"

## !IMPORTANT: This dictates how much ram will be used, and how much data will be loaded
# 1_280_000 loads around 5gb, dont push this too high as it will crash if ram deplects
# NUM_EXAMPLES_TO_LOAD_PER_FETCH = 1_280_000 
NUM_EXAMPLES_TO_LOAD_PER_FETCH = 640_000 
BATCH_SIZE = 1024

test = ChessEvalDataset(file = DATASET_PATH, model=TRAINING_MODE, load_batch_size = NUM_EXAMPLES_TO_LOAD_PER_FETCH)
loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CompleteChessBotNetwork().to(device)

# Continue with pretrained weights
model.load_state_dict(torch.load('./models/CompleteModel_new_architecture_skipmoves_epoch-30.pth'))
# model.load_state_dict(torch.load('./models/CompleteModel_noskip_version2_epoch-170.pth'))

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(torch.cuda.is_available())
print("Using device: ", device)
for epoch in range(7, NUM_EPOCHS+1):
    model.train()
    t0 = time()
    avg_loss = 0.0
    i = 1
    correct = 0
    for board_tensor, target_eval, valid_moves in tqdm.tqdm(loader):        
        board_tensor_gpu, target_eval_gpu = board_tensor.to(device), target_eval.to(device)  # Move data to GPU
        optimizer.zero_grad()
        pred = model(board_tensor_gpu)

        # Compute loss with valid move vlaidaiton
        # loss = loss_fn(board_tensor, pred, target_eval_gpu.squeeze(1))
        loss = loss_fn(pred, target_eval_gpu.squeeze(1), valid_moves)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        avg_loss += loss.item()
        i+=1
        correct += (pred.argmax(dim=1) == target_eval_gpu[:, 0]).sum().item()
    accuracy = 100 * correct / (len(loader) * BATCH_SIZE)
    tf = time()
    print(f"Epoch {epoch} - {avg_loss / len(loader):.4f} | Accuracy: {accuracy:.2f}% | Time: {tf-t0}")

    if epoch % 5 == 0:
        torch.save(model.state_dict(), f"./models/CompleteModel_new_architecture_skipmoves_epoch-{epoch}.pth")
# Save the trained model
torch.save(model.state_dict(), MODEL_WEIGHTS_OUTPUT_PATH)

# filtra 1100 - 14000

True
Using device:  cuda


100%|██████████| 2744/2744 [03:45<00:00, 12.18it/s]


Epoch 7 - 1.4678 | Accuracy: 52.09% | Time: 225.31490898132324


100%|██████████| 2744/2744 [03:44<00:00, 12.25it/s]


Epoch 8 - 1.4615 | Accuracy: 52.23% | Time: 224.08702063560486


...

Epoch 159 - 1.2825 | Accuracy: 57.04% | Time: 307.81230425834656


100%|██████████| 2744/2744 [05:08<00:00,  8.89it/s]  


Epoch 160 - 1.2820 | Accuracy: 57.05% | Time: 308.5661563873291


 97%|█████████▋| 2654/2744 [04:59<00:10,  8.86it/s]  


KeyboardInterrupt: 